In [0]:
!pip install pandas pymysql sqlalchemy

In [0]:
import re
import pandas as pd

class MySQLConnectnDataframe:

    def __init__(self, userName, userPassword, hostServer, dbName):
        from sqlalchemy import create_engine
        from sqlalchemy.pool import NullPool
        import pymysql
        pymysql.install_as_MySQLdb()
        import MySQLdb
        self.engine = create_engine(
                            'mysql+mysqldb://'
                            + userName + ':'
                            + userPassword + '@'
                            + hostServer + '/'
                            + dbName
                            + '?charset=utf8',
                            encoding='utf8',
                            poolclass=NullPool
        )

    def _appendDataframeAsRecordsToMySqlTable(self, dataFrame, tableName):
        #import pandas as pd
        conn = self.engine.connect()
        print('00 MySQL Database is connected.')
        df = dataFrame
        df.to_sql(con=conn, name=tableName, if_exists='append', index=False)
        print('00 %i Records has been inserted from Dataframe.' % len(dataFrame) )
        conn.close()
        print('00 MySQL Database is disconnected.')

    def _getDataframeFromQueryOnMySqlTable(self, query):
        from pandas import read_sql
        conn = self.engine.connect()
        print('00 MySQL Database is connected.')
        df = read_sql(query, con=conn)
        print('00 %i Records has been queried to Dataframe.' % len(df) )
        conn.close()
        print('00 MySQL Database is disconnected.')
        return(df)
    
    def _getDataframeFromHtml(self, url):
        from pandas import read_html
        df = read_html(url)
        return(df)

db = MySQLConnectnDataframe('<your-db-user>','<your-db-password>','<your-db-endpoint>','<your-db-databasename>')
res = db._getDataframeFromQueryOnMySqlTable('select caseserialnumber, casenumber, referenceprovision, casename from casetbl')

00 MySQL Database is connected.
00 81508 Records has been queried to Dataframe.
00 MySQL Database is disconnected.


In [0]:
# df = db._getDataframeFromHtml('http://www.scourt.go.kr/portal/information/event/guide/index14.html')
# db._appendDataframeAsRecordsToMySqlTable(df[0], 'casenumberLegend')

In [0]:
dfcolumn = ['caseIdx', 'caseyear', 'casetype', 'caseref', 'casename']
dfindex = list( range(len(res)) )
tempdf = pd.DataFrame(columns=dfcolumn, index=dfindex)
for rowidx in range( len(res) ):
    tempdf[dfcolumn[0]][rowidx] = res['caseserialnumber'][rowidx]   #소송년도
    tempdf[dfcolumn[1]][rowidx] = '|'.join( str(x) for x in set(re.findall( '([0-9]+)[가-힣]+[0-9]+', res['casenumber'][rowidx])) )   #소송번호
    tempdf[dfcolumn[2]][rowidx] = '|'.join( str(x) for x in set(re.findall( '[0-9]+([가-힣]+)[0-9]+', res['casenumber'][rowidx])) )   #사건구분
    #tempdf[dfcolumn[3]][rowidx] = '|'.join( str(x) for x in set( [x[0] for x in re.findall( '\n*\s*(대법원|[가-힣]+법|[가-힣][가-힣\s]+법|[가-힣]+령|[가-힣][가-젛,젝-힣\s]+령|[가-힣][가-젛,젝-힣\s]+칙|[가-힣][가-힣\s]+\s법률)(\s|\(|제)\s*', res['referenceprovision'][rowidx])] ) )    #참조볍령
    tempdf[dfcolumn[3]][rowidx] = '|'.join( str(x) for x in set( [x[0] for x in re.findall( '\n*\s*(대법원|[가-힣]+법)(\s|\(|제)\s*', res['referenceprovision'][rowidx])] ) )    #참조볍령
    tempdf[dfcolumn[4]][rowidx] = res['casename'][rowidx]   #소송명    

In [0]:
s = set()
for i in tempdf['caseref']:
    tmp = i.split('|')
    s.update(tmp)

In [0]:
legend = pd.DataFrame( data = list(s), columns = ['nameoflaw'] )
legend.head()

,nameoflaw
0,
1,산림법
2,군형법
3,도시계획법
4,건설업법


In [0]:
tempdf.head()

,caseIdx,caseyear,casetype,caseref,casename
0,0,,,,null
1,64437,2003,나,민법|파산법,사해행위취소등
2,64438,2008,도,형사소송법|형법,위증
3,64439,2007,무,인지법|민사소송법|행정소송법,상고장각하명령에대한이의
4,64440,2006,다,헌법|민법,손해배상(기)


In [0]:
tempdf[ tempdf['caseref'] == '' ].head()

,caseIdx,caseyear,casetype,caseref,casename
0,0,,,,null
16,64452,2008,다,,소유권말소등기
23,64459,2006,두,,시정조치및과징금납부명령취소
26,64462,2008,두,,국가유공자등록거부처분취소
27,64463,2008,두,,정보공개거부처분취소
